In [113]:
import pandas as pd
import numpy as np


#### Data Source
https://www.himalayandatabase.com/index.html

##### Guide
https://www.himalayandatabase.com/downloads/HD%20Online%20Guide.pdf

In [169]:
df_peaks = pd.read_csv("./Himalayan Database/RawData/peaksall.csv")
df_ex = pd.read_csv("./Himalayan Database/RawData/expeditions.csv")
df_mem = pd.read_csv("./Himalayan Database/RawData/members.csv")

C:\Users\botiv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\botiv\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [170]:
df_mem.columns

Index(['expid', 'membid', 'peakid', 'myear', 'mseason', 'fname', 'lname',
       'sex', 'age', 'birthdate', 'yob', 'calcage', 'citizen', 'status',
       'residence', 'occupation', 'leader', 'deputy', 'bconly', 'nottobc',
       'support', 'disabled', 'hired', 'sherpa', 'tibetan', 'msuccess',
       'mclaimed', 'mdisputed', 'msolo', 'mtraverse', 'mski', 'mparapente',
       'mspeed', 'mhighpt', 'mperhighpt', 'msmtdate1', 'msmtdate2',
       'msmtdate3', 'msmttime1', 'msmttime2', 'msmttime3', 'mroute1',
       'mroute2', 'mroute3', 'mascent1', 'mascent2', 'mascent3', 'mo2used',
       'mo2none', 'mo2climb', 'mo2descent', 'mo2sleep', 'mo2medical',
       'mo2note', 'death', 'deathdate', 'deathtime', 'deathtype', 'deathhgtm',
       'deathclass', 'msmtbid', 'msmtterm', 'hcn', 'mchksum', 'host', 'comrte',
       'stdrte', 'route1', 'route2', 'route3', 'route4', 'nation', 'leaders',
       'sponsor', 'termreason', 'totmembers', 'smtmembers', 'mdeaths',
       'tothired', 'nohired', 'smthire

### Countries

In [171]:
print(list(set(("/").join(df_mem.citizen.dropna().unique()).split("/"))))

['Bosnia-Herzegovina', 'Netherlands', 'Oman', 'Estonia', 'Dominican Republic', 'Costa Rica', 'Russia', 'Qatar', 'Zambia', 'Peru', 'Malaysia', 'Cyprus', 'Finland', 'Montenegro', 'Tunisia', 'Lebanon', 'Ecuador', 'Turkey', 'Bangladesh', 'Malta', 'Bolivia', 'Kuwait', 'Albania', 'Bahrain', 'Kyrgyz Republic', 'Taiwan', 'Denmark', 'Pakistan', 'Serbia', 'Tanzania', 'Kazakhstan', 'Liechtenstein', 'UK', 'Saudi Arabia', 'Chile', 'Switzerland', 'Bulgaria', 'Hungary', 'Belgium', 'Brazil', 'India', 'Thailand', 'Jordan', 'UAE', 'Sri Lanka', 'Uzbekistan', 'Poland', 'Portugal', 'Vietnam', 'Latvia', 'Morocco', 'Nepal', 'Myanmar', 'Mexico', 'Norway', 'Japan', 'S Africa', 'Colombia', 'Slovakia', 'Greece', 'Ireland', 'Indonesia', 'Lithuania', 'Panama', 'Armenia', 'Moldova', 'Andorra', 'China', 'Bhutan', 'Romania', 'San Marino', 'Yugoslavia', 'Guatemala', 'Luxembourg', 'Jamaica', 'Israel', 'USSR', 'Palestine', 'France', 'Uruguay', 'Hong Kong', 'Italy', 'Germany', 'Mauritius', 'Algeria', 'Kenya', 'Belarus', 

In [172]:
##df_mem.citizen.apply(lambda x : coco.convert(names=x, to='ISO2') if manual_map.get(x) == None  else manual_map.get(x))

### Get Members ready to join

In [173]:
df_mem['female']  = df_mem.sex.apply(lambda x : True if x=="F" else False)

In [174]:
df_mem.mspeed.value_counts()

False    65521
True        13
Name: mspeed, dtype: int64

In [175]:
gdf_mem = df_mem.groupby('peakid').agg({'expid' : 'count' ,'death' : 'sum', 'female' : 'sum', 'calcage' : 'mean', 'citizen' : 'unique','mo2used' : 'sum', "route1" : pd.Series.mode})

#### Grouby only the succeeding members

In [176]:
gdf_mem_success = df_mem[df_mem.msuccess == True].groupby('peakid').agg({'citizen' : 'unique', 'female' : 'sum','expid' : 'count' })

In [177]:
gdf_mem_success 

,citizen,female,expid
peakid,,,
ACHN,"[France, Nepal, Japan]",1.0,11
AMAD,"[W Germany, USA, Nepal, France, New Zealand, J...",322.0,3540
AMPG,"[UK, Nepal]",1.0,10
AMPH,[S Korea],1.0,3
AMPM,[Germany],1.0,5
...,...,...,...
URKM,"[Japan, Nepal, USA, Austria, France]",3.0,23
URMA,"[Taiwan, S Korea, Singapore, Australia, Nepal,...",11.0,52
YAKA,"[Japan, Nepal]",0.0,4


In [178]:
#gdf_mem.columns = ['_'.join(col).strip() for col in gdf_mem.columns.values]
#gdf_mem_success.columns = ['_'.join(col).strip() for col in gdf_mem_success.columns.values]

In [179]:
## Rename columns accordingly

In [180]:
gdf_mem.rename(columns={'expid' : 'count_mem_try', "female" :  "count_mem_try_female", "calcage" :"mean_mem_try_age", "citizen" : "country_mem_try", "route1" : "most_pop_route" }, inplace=True)
gdf_mem_success.rename(columns={'expid' : 'count_mem_success', "female" :  "count_mem_success_female", "calcage" :"mean_mem_success_age", "citizen" : "country_mem_success" }, inplace=True)

In [181]:
gdf_mem = pd.merge(gdf_mem,gdf_mem_success, on="peakid", how="left") 

In [182]:
gdf_mem.reset_index(inplace=True)

### Merge with Peak data

In [183]:
df = pd.merge(df_peaks,gdf_mem, on='peakid', how="left")

In [184]:
## WOrk with the ones with coords
df.dropna(subset=["lng"], inplace=True)

##### Adding ISO the the countries

In [185]:
import country_converter as coco

manual_map = {"USSR": 'USSR', "UK" : "GB", "UAE" : "AE", "W Germany" : "DE", "S Africa" : "ZA", "Yugoslavia" : "YUG" }


In [186]:
df.country_mem_success = df.country_mem_success.apply(lambda x : x if isinstance(x,float) else sum([i.split("/") for i in x ], []))

In [187]:
df.country_mem_success = df.country_mem_success.apply(lambda x : [ c if manual_map.get(c) == None else manual_map.get(c) for c in x ] if not isinstance(x,float) else x)

In [188]:
df.country_mem_success = df.country_mem_success.apply(lambda x : coco.convert(names=x, to='ISO2' , not_found=None) if not isinstance(x,float) else x)

YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex
YUG not found in ISO3
YUG not found in ISO3
USSR not found in regex


In [189]:
df.country_mem_success[~df.country_mem_success.isna()].apply(len)

2      99
4       2
8      40
9       7
10      9
       ..
446     2
451     3
452    11
456     2
457    11
Name: country_mem_success, Length: 131, dtype: int64

### Line Chart data create

In [190]:
df_temp = df_mem.groupby(['peakid', 'myear']).agg({'membid' : 'count','msuccess' : 'sum','death' : 'sum'})

In [191]:
df_temp = df_temp.reset_index().sort_values(["peakid", 'myear'], ascending=[True,True]).groupby(['peakid' ]).agg(list).rename(columns={"myear" : "lineX", "membid" : "lineY_attemps", "msuccess" : "lineY_success", "death" : "lineY_death"} )

In [192]:
df = pd.merge(df,df_temp, on='peakid', how="left")

In [193]:
df

,peakid,pkname,pkname2,location,heightm,heightf,himal,region,open,unlisted,...,country_mem_try,mo2used,most_pop_route,country_mem_success,count_mem_success_female,count_mem_success,lineX,lineY_attemps,lineY_success,lineY_death
0,AMAD,Ama Dablam,Amai Dablang,Khumbu Himal,6814,22356,12,2,True,False,...,"[France, USA, W Germany, Nepal, New Zealand, A...",15.0,SW Ridge,"[DE, US, NP, FR, NZ, JP, ES, AU, CH, AT, BE, K...",322.0,3540.0,"[1958, 1959, 1961, 1978, 1979, 1980, 1981, 198...","[6, 8, 5, 9, 54, 16, 35, 15, 46, 24, 54, 43, 5...","[0.0, 0.0, 4.0, 0.0, 26.0, 11.0, 17.0, 8.0, 33...","[0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, ..."
1,AMPG,Amphu Gyabjen,NaN,Khumbu Himal (N of Ama Dablam),5630,18471,12,2,True,False,...,"[UK, Nepal]",4.0,S Ridge,"[GB, NP]",1.0,10.0,"[1953, 1959, 2010]","[4, 6, 3]","[4.0, 6.0, 0.0]","[0.0, 0.0, 0.0]"
2,ANN1,Annapurna I,NaN,Annapurna Himal,8091,26545,1,5,True,False,...,"[W Germany, Iran, UK, USA, Nepal, Japan, Italy...",58.0,N Face,"[GB, NL, NP, CZ, US, JP, FR, DE, CH, AT, IT, P...",11.0,225.0,"[1950, 1961, 1965, 1969, 1970, 1973, 1975, 197...","[16, 1, 5, 11, 34, 25, 9, 13, 24, 32, 31, 39, ...","[2.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 4.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 1.0, 0.0, 2.0, ..."
3,ANN2,Annapurna II,NaN,Annapurna Himal,7937,26040,1,5,True,False,...,"[UK, India, Nepal, Yugoslavia, Japan, W German...",3.0,NW Ridge-W Ridge,"[GB, NP, YUG, JP, AU, KR, DE]",0.0,16.0,"[1950, 1953, 1957, 1960, 1969, 1971, 1973, 197...","[1, 2, 5, 19, 10, 12, 6, 8, 7, 3, 14, 28, 5, 2...","[0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,ANN3,Annapurna III,NaN,Annapurna Himal,7555,24787,1,5,True,False,...,"[Japan, Nepal, Italy, USA, UK, Australia, Swit...",5.0,N Face,"[JP, NP, IT, GB, CH, KR, IN, ES, US]",3.0,31.0,"[1961, 1970, 1977, 1978, 1979, 1980, 1981, 198...","[10, 11, 29, 15, 7, 24, 3, 21, 9, 4, 30, 13, 6...","[3.0, 4.0, 5.0, 0.0, 2.0, 2.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 1.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,TSUR,Tsaurabong Peak,"Chara Ban Peak, White Peak",Dhaulagiri Himal (SSE of Dhaulagiri V),6395,20981,4,6,False,True,...,"[Japan, Spain, Nepal]",0.0,E Ridge,"[JP, ES, NP]",0.0,30.0,"[1974, 1975, 1979, 1987]","[3, 15, 8, 6]","[3.0, 14.0, 7.0, 6.0]","[0.0, 0.0, 0.0, 0.0]"
147,TUKU,Tukuche,NaN,Dhaulagiri Himal (NE of Dhaulagiri I),6920,22703,4,6,True,False,...,"[Switzerland, Nepal, Japan, Italy, France, Bel...",3.0,NW Ridge,"[JP, NP, KR, ES, GR, FR, US, NL, AT, SI, FI]",8.0,93.0,"[1968, 1969, 1970, 1972, 1974, 1976, 1977, 197...","[3, 6, 17, 6, 5, 10, 4, 16, 7, 11, 10, 17, 13,...","[3.0, 0.0, 3.0, 2.0, 4.0, 9.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
148,YAKA,Yakawa Kang,NaN,Damodar Himal (N of Thorong La),6482,21266,3,5,True,False,...,"[Japan, Nepal]",0.0,S Face,"[JP, NP]",0.0,4.0,"[2008, 2009, 2010]","[16, 2, 5]","[0.0, 0.0, 4.0]","[0.0, 0.0, 0.0]"
149,YALU,Yalung Kang,Yalungkar,Kangchenjunga Himal (W summit of Kangchenjunga),8505,27904,9,1,True,False,...,"[Japan, Austria, W Germany, Spain, Mexico, Aus...",53.0,SE Face,"[JP, DE, AT, MX, PL, FR, YUG, ES, KR, NP, USSR]",1.0,53.0,"[1973, 1974, 1975, 1978, 1980, 1981, 1984, 198...","[16, 13, 12, 14, 13, 24, 17, 16, 7, 17, 34, 6,...","[2.0, 0.0, 9.0, 0.0, 1.0, 5.0, 8.0, 2.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, ..."


In [194]:
df.to_json("./Himalayan Database/GeoData/peaks2.json", orient='records')

In [195]:
df.to_csv("./Himalayan Database/GeoData/peaks2.csv")

In [196]:
df['id'] = df.apply(lambda x : [x.lat,x.lng], axis=1)
df['label'] = df.pkname + " - " + df.heightm.astype("str") + "m"

In [197]:
df[['id', 'label']].to_json("./Himalayan Database/GeoData/peaks_search.json" ,orient="records")

In [198]:
geojson = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [d["lon"], d["lat"]],
            },
        "properties" : d,
     } for d in data]
}

NameError: name 'data' is not defined

In [ ]:
df.to_dict(orient="records")